In [1]:
import os
from ranger import Ranger

import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import Normalize
from torchmetrics import Accuracy

import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc

import warnings
warnings.filterwarnings('ignore')

In [2]:
print(torch.cuda.is_available())

True


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
batch_size = 8
epochs = 100
train_path = '../datasets/imagenette2/train'
val_path = '../datasets/imagenette2/val'


In [10]:
train_dataloader = DataLoader(datasets.ImageFolder(train_path, transform = transforms.Compose([
                                                                    transforms.RandomResizedCrop(224),
                                                                    transforms.RandomHorizontalFlip(),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                            ])), batch_size = batch_size, shuffle=True)

test_dataloader = DataLoader(datasets.ImageFolder(val_path,
                                                               transform=transforms.Compose([
                                                                   transforms.ToTensor(),
                                                                   transforms.Resize([224, 224]),
                                                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                               ])),batch_size=batch_size, shuffle=True)

In [11]:
classes = ('tench', 'springer', 'casette_player', 'chain_saw','church', 'French_horn', 'garbage_truck', 'gas_pump', 'golf_ball', 'parachute')


In [13]:
%run models.ipynb
%run utils.ipynb

In [16]:
model_normal = resnet18_features(pretrained=False, filter='None', filter_layer=0)
learning_rate = 1e-04
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()
optimizer = Ranger(model_normal.parameters(), lr = learning_rate, eps = 1e-06)
print(device)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
cuda


In [17]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [18]:
model_normal = train_model(model = model_normal.to(device), epochs = epochs)

Epoch 1/100 - test accuracy: 54.34% and CE loss 1.32
Epoch 2/100 - test accuracy: 61.53% and CE loss 0.66
Epoch 3/100 - test accuracy: 67.62% and CE loss 0.75
Epoch 4/100 - test accuracy: 69.73% and CE loss 0.99
Epoch 5/100 - test accuracy: 71.67% and CE loss 0.36
Epoch 6/100 - test accuracy: 73.78% and CE loss 0.42
Epoch 7/100 - test accuracy: 73.94% and CE loss 0.81
Epoch 8/100 - test accuracy: 74.24% and CE loss 0.61
Epoch 9/100 - test accuracy: 74.52% and CE loss 0.97
Epoch 10/100 - test accuracy: 76.66% and CE loss 0.18
Epoch 11/100 - test accuracy: 78.47% and CE loss 1.21
Epoch 12/100 - test accuracy: 77.43% and CE loss 1.17
Epoch 13/100 - test accuracy: 79.03% and CE loss 0.22
Epoch 14/100 - test accuracy: 79.90% and CE loss 0.11
Epoch 15/100 - test accuracy: 78.52% and CE loss 0.66
Epoch 16/100 - test accuracy: 79.31% and CE loss 0.60
Epoch 17/100 - test accuracy: 80.59% and CE loss 0.08
Epoch 18/100 - test accuracy: 81.68% and CE loss 0.43
Epoch 19/100 - test accuracy: 80.71% 

In [19]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 9.07%


In [20]:
from pathlib import Path

MODEL_PATH = Path("../models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "resnet_imagenette.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_normal.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: ..\models\resnet_imagenette.pth
